In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/minsir/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "minsirx.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
os.environ["OPENAI_API_KEY"] = "sk-proj-FZ2v2sIScsxguqHnuT4VT3BlbkFJEgAuDrqMx91v7PG2fhDu"
django.setup()
from apps.email_manager.service_layer.email_linker import ConversationReader
import json
from datetime import datetime
from apps.email_manager.service_layer.insurance_erp_llm import InsuranceERPLLM
from apps.email_manager.models.email import EmailAttachment



In [2]:
convo_reader = ConversationReader(
    start_date=datetime(year=2024, month=2, day=10),
    end_date=datetime(year=2024, month=2, day=20),
)
convo_reader.extract_conversation()


/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/django/db/models/fields/__init__.py:1659: RuntimeWarning: DateTimeField EmailData.date received a naive datetime (2024-02-10 00:00:00) while time zone support is active.
  warnings.warn(
/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/django/db/models/fields/__init__.py:1659: RuntimeWarning: DateTimeField EmailData.date received a naive datetime (2024-02-20 00:00:00) while time zone support is active.
  warnings.warn(


In [6]:
for a_convo_str, a_convo_data in convo_reader.conversations.items():
    if 'ISSUE POLICY FOR EEI | M/S SIKA PAKISTAN ' in a_convo_str:
        (full_convo, attachments) = a_convo_data.get_full_convo()
        attachments: list[EmailAttachment] = attachments
        insLLM = InsuranceERPLLM()
        # insLLM.store_documents_in_vector_store(attachments)
        # insLLM.search_documents_vector_store()
        # insLLM.query_from_final_results(insLLM.document_query_results)
        insLLM.store_convo_in_vector_store(f"{full_convo}")
        insLLM.search_convo_vector_store()
        insLLM.query_from_final_results(insLLM.convo_query_results)
        # for attchmnt in attachments:
        #     print(attchmnt)
        # for a_msg in full_convo:
        #     print(json.dumps(a_msg, indent=4))
        print("")
        print("------------------------------------")
        print("")
        break


{
    "net_premium": {
        "Choice 1": "[{'from': 'afzal.qadir@adamjeeinsurance.com', 'to': ['mufzal.agha@adamjeeinsurance.com'], 'date': 'February 19, 2024, 6:33 AM', 'subject': 'ISSUE POLICY FOR EEI | M/S SIKA PAKISTAN (PVT.) LTD.', 'body': 'Dear Agha,\\n\\xa0\\nPlease issue policy for EEI as per payment instrument attached.\\n\\xa0\\n\\xa0 Afzal Qadir\\nTeam Member,\\nMiscellaneous Underwriting\\nP\\xa0\\n:\\n\\xa0\\xa0+92 21 32412623\\xa0\\n\\xa0Ext \\n1370 , \\nM\\xa0\\n: +92",
        "Choice 2": "Underwriting\\nP\\xa0\\n:\\n\\xa0\\xa0+92 21 32412623\\xa0\\n\\xa0Ext \\n1370 , \\nM\\xa0\\n: +92 3350203993\\xa0\\xa0\\nE\\xa0\\n:\\xa0\\nafzal.qadir\\n@adamjeeinsurance.com\\nAdamjee Insurance Company Limited\\nAdamjee House Division, 5th Floor, \\nAdamjee House, I.I.Chundrigar Road, Karachi,\\n\\xa0\\xa0\\xa0\\xa0\\xa0\\xa0\\xa0\\xa0\\xa0\\xa0\\xa0\\xa0\\xa0\\xa0\\xa0\\xa0\\xa0\\xa0\\xa0\\xa0\\xa0\\xa0 Pakistan.\\xa0\\xa0\\xa0\\xa0 \\n\\xa0\\xa0\\xa0\\xa0\\xa0\\n\\xa0\\xa0\\xa0\\

In [5]:
for a_convo_str, a_convo_data in convo_reader.conversations.items():
    if 'ISSUE POLICY FOR EEI | M/S SIKA PAKISTAN ' in a_convo_str:
        (full_convo, attachments) = a_convo_data.get_full_convo()
        attachments: list[EmailAttachment] = attachments
        # insLLM = InsuranceERPLLM()
        # insLLM.store_documents_in_vector_store(attachments)
        # insLLM.search_documents_vector_store()
        # insLLM.query_from_final_results(insLLM.document_query_results)
        for attchmnt in attachments:
            print(attchmnt)
        for a_msg in full_convo:
            print(json.dumps(a_msg, indent=4))
        print("")
        print("------------------------------------")
        print("")
        break

/Users/mirbilal/Desktop/minsir/media/email_attachments/image008_m7Y2kg4.png
/Users/mirbilal/Desktop/minsir/media/email_attachments/image002_Xhg4ov5.jpg
/Users/mirbilal/Desktop/minsir/media/email_attachments/image003_LfHcqxg.jpg
/Users/mirbilal/Desktop/minsir/media/email_attachments/image004_xoircVT.jpg
/Users/mirbilal/Desktop/minsir/media/email_attachments/image005_MLVoby1.jpg
/Users/mirbilal/Desktop/minsir/media/email_attachments/image006_oVIYU8F.png
/Users/mirbilal/Desktop/minsir/media/email_attachments/image001_9ZGV6ei.jpg
/Users/mirbilal/Desktop/minsir/media/email_attachments/image007_3pUWBQ0.jpg
/Users/mirbilal/Desktop/minsir/media/email_attachments/image009_rjg8YEX.jpg
/Users/mirbilal/Desktop/minsir/media/email_attachments/image010_4lMZ3rT.png
/Users/mirbilal/Desktop/minsir/media/email_attachments/ADAMJEE_INSURANCE_PAYMENT_SWIFT_16.02.2024-2_ZrruCMA.pdf
/Users/mirbilal/Desktop/minsir/media/email_attachments/PL-0123-202504-E12-000001_-_LAPTOPS_5PTg6tE.pdf
/Users/mirbilal/Desktop/m

In [ ]:
insLLM = InsuranceERPLLM()
insLLM.store_convo_in_vector_store(f"{dummy_convo}")
insLLM.search_convo_vector_store()
insLLM.query_from_final_results(insLLM.convo_query_results)